<a href="https://colab.research.google.com/github/kimbanica/Agente-Aut-nomo-de-An-lise-Fiscal-v1.0-/blob/main/Agente_Aut%C3%B4nomo_de_An%C3%A1lise_Fiscal_(v1_0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CÉLULA 1: Instalações
print("Instalando dependências (OpenAI, Google/Gemini, Gradio, Langchain)...")
!pip install gradio pandas langchain langchain-experimental langchain-community llama-cpp-python openai tabulate python-multipart langchain-google-genai

In [ ]:
# CÉLULA 2: Leitura Segura das Chaves de API
import os
from google.colab import userdata

# Tenta ler as chaves do Cofre de Senhas
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

    if not os.environ["OPENAI_API_KEY"]:
        print("ATENÇÃO: Chave 'OPENAI_API_KEY' não encontrada no Cofre. Verifique o nome e se o acesso ao notebook está ligado.")

    if not os.environ["GOOGLE_API_KEY"]:
        print("ATENÇÃO: Chave 'GOOGLE_API_KEY' não encontrada no Cofre. Verifique o nome e se o acesso ao notebook está ligado.")

    print("Chaves de API carregadas do Cofre com sucesso.")

except Exception as e:
    print(f"Erro ao acessar o Cofre de Senhas: {e}")
    print("Por favor, verifique se você adicionou as chaves 'OPENAI_API_KEY' e 'GOOGLE_API_KEY' no menu 🔑 'Secrets' à esquerda.")

In [ ]:

# CÉLULA 3: Criação do Arquivo do Agente (Backend)
# O comando %%writefile DEVE ser a primeira linha desta célula
%%writefile agente_csv.py
import os
import zipfile
import tempfile
import pandas as pd
import traceback

from dotenv import load_dotenv
load_dotenv()

# Importando os LLMs de AMBOS os provedores
from langchain_community.llms import OpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.agents import create_pandas_dataframe_agent

# Lendo as duas chaves do ambiente (que a Célula 2 configurou)
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

def get_prompt_especializado(ramo_atividade: str) -> str:
    """
    Cria o prompt com regras de negócio específicas.
    """
    prompt_especialista = f"O ramo de atividade da empresa é: **{ramo_atividade}**.\n"

    if ramo_atividade == "Indústria":
        prompt_especialista += "REGRAS DE INDÚSTRIA: Preste atenção em 'IPI' e 'Substituição Tributária'.\n"
    elif ramo_atividade == "Agronegócio":
        prompt_especialista += "REGRAS DE AGRONEGÓCIO: Monitore CFOPs específicos do setor (Venda de produção, Insumos).\n"
    elif ramo_atividade == "Setor Automotivo":
        prompt_especialista += "REGRAS DO SETOR AUTOMOTIVO: Valide notas fiscais de peças e serviços, confira códigos.\n"
    elif ramo_atividade in ["Órgãos Públicos", "Terceiro Setor"]:
         prompt_especialista += f"REGRAS DE {ramo_atividade.upper()}: A análise de 'Centros de Custo' é prioritária.\n"
    else:
        prompt_especialista += "Nenhuma regra específica de ramo. Siga as instruções gerais.\n"

    return prompt_especialista


def responder(arquivo_zip_path: str, pergunta_usuario: str, ramo_atividade: str) -> str:
    """
    Função principal do agente, agora com lógica de Fallback:
    1. Tenta OpenAI
    2. Se falhar, tenta Gemini
    """

    # --- 1. Preparação (Leitura do CSV) ---
    try:
        temp_dir = tempfile.mkdtemp()
        with zipfile.ZipFile(arquivo_zip_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)

        csv_files = [os.path.join(temp_dir, f) for f in os.listdir(temp_dir) if f.endswith('.csv')]
        if not csv_files:
            return "Nenhum arquivo CSV encontrado no ZIP."

        caminho_csv = csv_files[0]

        # --- LÓGICA DE LEITURA ROBUSTA (MODIFICADO) ---
        try:
            # 1. Tenta o padrão (vírgula, engine C, utf-8)
            df = pd.read_csv(caminho_csv, encoding='utf-8')
        except (pd.errors.ParserError, UnicodeDecodeError) as e:
            print(f"Falha na leitura padrão (vírgula): {e}. Tentando com ponto-e-vírgula...")
            try:
                # 2. Tenta com ponto-e-vírgula (comum no Brasil, latin1)
                df = pd.read_csv(caminho_csv, sep=';', encoding='latin1')
            except pd.errors.ParserError as e2:
                print(f"Falha com ponto-e-vírgula: {e2}. Tentando pular linhas ruins...")
                try:
                    # 3. Tenta pular linhas ruins (engine python)
                    # on_bad_lines='skip' é a chave aqui
                    df = pd.read_csv(caminho_csv, encoding='latin1', engine='python', on_bad_lines='skip')
                    print(f"Aviso: O CSV foi lido pulando linhas mal formatadas.")
                except Exception as e_final:
                    # 4. Falha total
                    print(f"Erro final de leitura: {e_final}")
                    return f"Erro Crítico ao ler o CSV: O arquivo parece estar corrompido ou em formato desconhecido. Detalhe: {str(e_final)}"
        # --- FIM DA LÓGICA DE LEITURA ROBUSTA ---

    except Exception as e:
        # Pega erros do ZIP ou outros erros de arquivo
        return f"Erro ao extrair o ZIP ou encontrar o CSV: {str(e)}"

    # --- 2. Construção do Cérebro (Prompt) ---
    prefixo_base = "Você é um assistente de dados avançado que fala apenas português. Você é especialista em analisar DataFrames do Pandas."
    prompt_especialista = get_prompt_especializado(ramo_atividade)
    prefixo_final = prefixo_base + "\n" + prompt_especialista

    pergunta_para_agente = (
        "Tarefas:\n"
        "1. (TAREFA PRINCIPAL): Se as colunas 'Tipo_Documento' e 'Centro_Custo' não existirem, crie-as e classifique CADA LINHA por tipo (compra, venda, serviço) e por centros de custos.\n"
        "2. (TAREFA DO USUÁRIO): Após a classificação, responda a esta pergunta: "
        f"'{pergunta_usuario}'\n"
        "Responda ao usuário em português."
    )

    # --- 3. Lógica de Execução (OpenAI com Fallback para Gemini) ---

    # TENTATIVA 1: OpenAI (O "Engenheiro" preferido)
    try:
        if not openai_api_key:
            raise ValueError("OPENAI_API_KEY não foi carregada do Cofre de Senhas.")

        print("Iniciando Tentativa 1 (OpenAI)...")
        llm_openai = OpenAI(temperature=0, openai_api_key=openai_api_key)

        agente_openai = create_pandas_dataframe_agent(
            llm_openai, df, prefix=prefixo_final, verbose=True,
            allow_dangerous_code=True, agent_executor_kwargs={"handle_parsing_errors": True}
        )

        resposta = agente_openai.run(pergunta_para_agente)
        print("Resposta obtida com OpenAI.")
        return f"**(Respondido por: OpenAI)**\n\n" + str(resposta)

    except Exception as e_openai:
        print(f"Erro com OpenAI: {str(e_openai)}")
        print("Iniciando Tentativa 2 (Fallback com Gemini)...")

        # TENTATIVA 2: Gemini (O "Estrategista" de backup)
        try:
            if not google_api_key:
                raise ValueError("GOOGLE_API_KEY não foi carregada do Cofre de Senhas para o fallback.")

            llm_gemini = ChatGoogleGenerativeAI(
                model="gemini-1.5-pro-latest", # Recomendo 1.5 Pro
                temperature=0,
                google_api_key=google_api_key
            )

            agente_gemini = create_pandas_dataframe_agent(
                llm_gemini, df, prefix=prefixo_final, verbose=True,
                allow_dangerous_code=True, agent_executor_kwargs={"handle_parsing_errors": True}
            )

            resposta = agente_gemini.run(pergunta_para_agente)
            print("Resposta obtida com Gemini (Fallback).")
            return f"**(Respondido por: Gemini 1.5 Pro - Fallback)**\n\n" + str(resposta)

        except Exception as e_gemini:
            print(f"Erro com Gemini (Fallback): {str(e_gemini)}")
            return (
                f"**Erro Crítico: Ambos os agentes falharam.**\n\n"
                f"**Erro OpenAI:** {str(e_openai)}\n\n"
                f"**Erro Gemini (Fallback):** {str(e_gemini)}"
            )

In [ ]:
# CÉLULA 4: Importação e Lançamento do Frontend (Gradio)
import gradio as gr
import agente_csv

print("\nDependências instaladas e agente_csv.py (com Fallback) criado.")
print("Iniciando a interface Gradio...")

# Função ponte (sem mudanças)
def executar_agente_interface(arquivo_zip, ramo_atividade, pergunta_usuario):
    if arquivo_zip is None:
        return "Erro: Por favor, faça o upload de um arquivo .ZIP."
    if not pergunta_usuario:
        return "Erro: Por favor, digite uma pergunta."

    caminho_zip = arquivo_zip.name
    resposta_do_agente = agente_csv.responder(caminho_zip, pergunta_usuario, ramo_atividade)
    return resposta_do_agente

# Ramos de atividade (sem mudanças)
ramos_de_atividade = [
    "Indústria", "Agronegócio", "Setor Automotivo", "Serviços",
    "Órgãos Públicos", "Terceiro Setor", "Varejo", "Outro"
]

# Construção da UI (sem mudanças)
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 Agente Autônomo para Análise de Documentos Fiscais")
    gr.Markdown("Faça o upload do seu arquivo **.ZIP**, selecione o ramo e faça sua pergunta.")

    with gr.Row():
        with gr.Column(scale=1):
            in_arquivo = gr.File(label="Upload do Arquivo .ZIP", file_types=[".zip"])
            in_ramo = gr.Dropdown(label="Selecione o Ramo de Atividade (Contexto)", choices=ramos_de_atividade, value="Indústria")
            in_pergunta = gr.Textbox(label="Digite sua pergunta sobre os dados", placeholder="Ex: Qual o valor total das notas fiscais de compra?")
            btn_processar = gr.Button("Perguntar ao Agente", variant="primary")
        with gr.Column(scale=2):
            out_resposta = gr.Markdown(label="Resposta do Agente")

    btn_processar.click(
        fn=executar_agente_interface,
        inputs=[in_arquivo, in_ramo, in_pergunta],
        outputs=[out_resposta]
    )

    gr.Markdown("---")
    gr.Markdown("### O agente tentará responder com OpenAI (GPT-4) e usará o Gemini 1.5 Pro como fallback.")

# 5. Lançar a interface
demo.launch(share=True, debug=True)